<h2><font color="#004D7F" size=5>Módulo 1: Modelos múltiples</font></h2>



<h1><font color="#004D7F" size=6> 3. Modelos de regresión de múltiples salidas </font></h1>

<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Manuel Castillo-Cara</font><br>
<font color="#004D7F" size=3>Aprendizaje Automático II</font><br>
<font color="#004D7F" size=3>Universidad Nacional de Educación a Distancia</font>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>


* [1. El problema de la regresión multisalida](#section1)
   * [1.1. Dataset](#section11)
* [2. Algoritmos de regresión inherentemente multisalida](#section2)
    * [2.1. Regresión lineal para regresión multisalida](#section21)
    * [2.2. KNN para regresión multisalida](#section22)
    * [2.3. Árbol de decisión para regresión multisalida](#section23)
    * [2.4. Evaluar la regresión de múltiples salidas con validación cruzada](#section24)
* [3. Wrapper para algoritmos de regresión multisalida](#section3)
* [4. Regresión multisalida directa](#section4)
    * [4.1. Evaluación](#section41)
    * [4.2. Predicción](#section42)
* [5. Regresión multisalida encadenada](#section5)
    * [5.1. Evaluación](#section51)
    * [5.2. Predicción](#section52)
* [Ejercicios](#sectionEj)

---

<a id="section0"></a>
# <font color="#004D7F">0. Contexto</font>

La regresión de salidas múltiples son problemas de regresión que implican predecir dos o más valores numéricos dado un ejemplo de entrada. Un ejemplo podría ser predecir unas coordenadas dada una entrada, por ejemplo, predecir los valores _x_ e _y_. 

Algunos algoritmos admiten inherentemente la regresión de múltiples salidas, como la regresión lineal y los árboles de decisión. También existen modelos de solución especiales que se pueden utilizar para ajustar y utilizar aquellos algoritmos que no admiten de forma nativa la predicción de múltiples resultados. En este tutorial, descubrirá cómo desarrollar modelos de aprendizaje automático para regresión de múltiples salidas, específicamente:
- El problema de la regresión multisalida.
- Cómo desarrollar modelos de aprendizaje automático que admitan inherentemente regresiones de múltiples salidas.
- Cómo desarrollar modelos conjuntos que permitan utilizar algoritmos que no soportan inherentemente múltiples resultados para la regresión de múltiples resultados.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section1"></a>
# <font color="#004D7F"> 1. El problema de la regresión multisalida</font>

Tenemos en regresión dos posibles problemas:
- __Regresión__: predecir una única salida numérica dada una entrada.
- __Regresión de múltiples salidas__: predecir dos o más salidas numéricas dada una entrada.

<figure><center>
  <img src="datos/multiclass.png" width="650" height="650" alt="Gráfica">
  <figcaption><blockquote>Gráfico con los diferentes algoritmos multiclase y multioutput en Sklearn. Extraída de la documentación de Scikit-learn</blockquote></figcaption>
</center></figure>

En la regresión de múltiples salidas, normalmente las salidas dependen de la entrada entre sí. Esto significa que a menudo los resultados no son independientes entre sí y pueden requerir un modelo que prediga ambos resultados juntos o que cada resultado dependa de los demás. 

<figure><center>
  <img src="datos/multioutput-regression2.png" width="450" height="450" alt="Gráfica">
  <figcaption><blockquote>Gráfico esquemático de los tipos de regresión multisalida que existen. Extraída de Geeksforgeeks.org</blockquote></figcaption>
</center></figure>

---

<figure><center>
  <img src="datos/multioutput-regression.png" width="450" height="450" alt="Gráfica">
  <figcaption><blockquote>Gráfico esquemático de Multisalida en regresión en el que un modelo predice para cada salida. Extraída de Geeksforgeeks.org</blockquote></figcaption>
</center></figure>

---

<figure><center>
  <img src="datos/multioutput-regression3.png" width="450" height="450" alt="Gráfica">
  <figcaption><blockquote>Gráfico esquemático de Multisalida en regresión en el que se entrenada cada modelo independientemente para predecir su salida correspondiente. Extraída de Geeksforgeeks.org</blockquote></figcaption>
</center></figure>

<a id="section11"></a>
## <font color="#004D7F"> 1.1. Dataset</font>

Usaremos la función `make_regression()` para crear un dataset 1000 ejemplos con 10 características de entrada, 5 de las cuales serán redundantes y 5 serán informativas. El problema requerirá la predicción de dos valores numéricos.
- Entrada del problema: 10 variables numéricas.
- Salida del problema: 2 variables numéricas.

In [1]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
print(X.shape, y.shape)

(1000, 10) (1000, 2)


In [2]:
X[400]

array([-0.64088429,  0.46455086,  0.55353581, -0.62778801, -0.3466824 ,
        2.97578929,  0.35570561, -0.26769419,  0.58943853, -0.18435229])

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section2"></a> 
# <font color="#004D7F"> 2. Algoritmos de regresión inherentemente multisalida </font>

Algunos algoritmos de aprendizaje automático de regresión admiten múltiples salidas directamente, como:
`LinearRegression` (y relacionados), `KNeighborsRegressor` y `DecisionTreeRegressor`.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre las clases [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html), [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn-neighbors-kneighborsregressor) y [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn-tree-decisiontreeregressor).
</div>

<a id="section21"></a> 
## <font color="#004D7F"> 2.1. Regresión lineal para regresión multisalida</font>

El siguiente ejemplo ajusta un modelo de regresión lineal en el conjunto de datos de regresión de múltiples salidas y luego realiza una predicción única con el modelo de ajuste.

In [3]:
from sklearn.linear_model import LinearRegression

# definir el modelo
model = LinearRegression()
# Entrenar
model.fit(X, y)
# Hacer la predicción
row = [-0.64088429,  0.46455086,  0.55353581, -0.62778801, -0.3466824 ,
        2.97578929,  0.35570561, -0.26769419,  0.58943853, -0.18435229]
yhat = model.predict([row])
print(yhat[0])

[-81.79958291  45.56655262]


<a id="section22"></a> 
## <font color="#004D7F"> 2.2. KNN para regresión multisalida</font>

El siguiente ejemplo ajusta un modelo KNN en el conjunto de datos de regresión de múltiples salidas y luego realiza una predicción única con el modelo de ajuste.

In [4]:
# k-nearest neighbors for multioutput regression
from sklearn.neighbors import KNeighborsRegressor

# definir el modelo
model = KNeighborsRegressor()
# Entrenar
model.fit(X, y)
# Hacer la predicción
row = [-0.64088429,  0.46455086,  0.55353581, -0.62778801, -0.3466824 ,
        2.97578929,  0.35570561, -0.26769419,  0.58943853, -0.18435229]
yhat = model.predict([row])
print(yhat[0])

[-45.05036526  64.63843193]


<a id="section23"></a> 
## <font color="#004D7F"> 2.3. Árbol de decisión para regresión multisalida</font>

El siguiente ejemplo ajusta un modelo Árbol de decisión en el conjunto de datos de regresión de múltiples salidas y luego realiza una predicción única con el modelo de ajuste.

In [5]:
# decision tree for multioutput regression
from sklearn.tree import DecisionTreeRegressor

# definir el modelo
model = DecisionTreeRegressor()
# Entrenar
model.fit(X, y)
# Hacer la predicción
row = [-0.64088429,  0.46455086,  0.55353581, -0.62778801, -0.3466824 ,
        2.97578929,  0.35570561, -0.26769419,  0.58943853, -0.18435229]
yhat = model.predict([row])
print(yhat[0])

[-82.3122973  45.4810737]


<a id="section24"></a> 
## <font color="#004D7F"> 2.4. Evaluar la regresión de múltiples salidas con validación cruzada</font>

Evaluamos una regresión de múltiples salidas utilizando una validación cruzada de _k_ veces.
- Ajustaremos y evaluaremos `DecisionTreeRegressor` utilizando una validación cruzada de 10 veces con 3 repeticiones. 
- Usaremos la métrica de rendimiento del error absoluto medio (MAE) como puntuación.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre [`cross_val_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre el parámetro [`scoring`](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

</div>

In [6]:
from numpy import mean
from numpy import std
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

# definir el modelo
# model = DecisionTreeRegressor()
# model = KNeighborsRegressor()
model = LinearRegression()
# definir el procedimiento de evaluación
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluar el modelo y recoger las puntuaciones
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# resumen de la puntuación
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -0.407 (0.020)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: La API de scikit-learn invierte el signo del MAE para transformarlo, de minimizar el error a maximizar el error negativo. Esto significa que los errores positivos de gran magnitud se convierten en grandes errores negativos (por ejemplo, 100 se convierte en -100) y un modelo perfecto no tiene ningún error con un valor de 0,0. También significa que podemos ignorar con seguridad el signo de las puntuaciones MAE medias. 

</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section3"></a> 
# <font color="#004D7F"> 3. Wrapper para algoritmos de regresión multisalida</font>

No todos los algoritmos de regresión admiten la regresión de múltiples salidas. Un ejemplo es la máquina de vectores de soporte, aunque para la regresión se la conoce como SVR. 

Una solución alternativa es dividir el problema de regresión de múltiples salidas en múltiples subproblemas. Por ejemplo, si un problema de regresión de múltiples salidas requiriera la predicción de 3 valores $y_1$, $y_2$ e $y_3$ dada una entrada $X$, entonces sería:
- Problema 1: Dado $X$, predice $y_1$.
- Problema 2: Dado $X$, predice $y_2$.
- Problema 3: Dado $X$, predice $y_3$.

Hay dos enfoques principales para implementar esta técnica. 
1. __Multisalida Directa__: Desarrollar un modelo independiente para cada valor numérico a ser predicho.
     - Implica desarrollar un modelo de regresión separado para cada valor de producción que se va a predecir. 
     - Podemos considerar esto como un enfoque directo, ya que cada valor objetivo se modela directamente.
     
2. __Multisalida encadenada__: Desarrollar una secuencia de modelos dependientes para que coincida con el número de valores numéricos a predecir.
    - La predicción del primer modelo se toma como parte de la entrada del segundo modelo, y el proceso de dependencia de salida a entrada se repite a lo largo de la cadena de modelos.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section4"></a> 
# <font color="#004D7F"> 4. Regresión multisalida directa</font>

El enfoque directo de la regresión de múltiples resultados implica dividir el problema de regresión en un problema separado para cada variable objetivo que se va a predecir. Esto supone que las salidas son independientes entre sí, lo que podría no ser una suposición correcta. 

Este enfoque está respaldado por la clase `MultiOutputRegressor` que toma un modelo de regresión como argumento. Luego creará una instancia del modelo proporcionado para cada resultado del problema. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre [`MultiOutputRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputRegressor.html)

</div>

<a id="section41"></a> 
## <font color="#004D7F"> 4.1. Evaluación</font>

Creamos primero un modelo de regresión de salida única y luego usar la clase `MultiOutputRegressor` para ajustar el modelo de regresión y agregar soporte para la regresión de múltiples salidas.

In [9]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR

# definir modelo base
model = LinearSVR()
# definir el modelo contenedor directo de salida múltiple
wrapper = MultiOutputRegressor(model)
# definir el procedimiento de evaluación
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluar el modelo y recoger las puntuaciones
n_scores = cross_val_score(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -0.419 (0.024)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section42"></a> 
## <font color="#004D7F"> 4.2. Predicción</font>

También podemos utilizar el contenedor de regresión directa de múltiples salidas como modelo final y hacer predicciones sobre nuevos datos. 
1. Primero, el modelo se ajusta a todos los datos disponibles, 
2. Luego se puede llamar a la función `predict()` para hacer predicciones sobre nuevos datos. 

El siguiente ejemplo demuestra esto en nuestro conjunto de datos de regresión sintética de múltiples salidas.

In [10]:
# definir modelo base
model = LinearSVR()
# definir el modelo contenedor directo de salida múltiple
wrapper = MultiOutputRegressor(model)
# ajustar el modelo en todo el conjunto de datos
wrapper.fit(X, y)
# hacer una sola predicción
row = [-0.64088429,  0.46455086,  0.55353581, -0.62778801, -0.3466824 ,
        2.97578929,  0.35570561, -0.26769419,  0.58943853, -0.18435229]
yhat = wrapper.predict([row])
print('Predicha: %s' % yhat[0])

Predicha: [-81.731962    45.55227737]


La ejecución del ejemplo ajusta el modelo contenedor directo en todo el conjunto de datos y luego se usa para hacer una predicción en una nueva fila de datos, como lo haríamos cuando usamos el modelo en una aplicación.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section5"></a> 
# <font color="#004D7F"> 5. Regresión multisalida encadenada</font>

Otro método es crear una secuencia lineal de modelos.  Por ejemplo, si un problema de regresión de múltiples salidas requiriera la predicción de 3 valores $y_1$, $y_2$ e $y_3$ dada una entrada $X$, entonces esto podría dividirse en tres problemas de regresión de salida única dependientes:
- __Problema 1__: Dado $X$, predice $y_1$.
- __Problema 2__: Dados $X$ e $yhat_1$, predice $y_2$.
- __Problema 3__: Dados $X$, $yhat_1$ y $yhat_2$, predice $y_3$.

Esto se puede lograr utilizando la clase `RegressorChain` en la biblioteca scikit-learn. El orden de los modelos
puede basarse en el orden de las salidas en el conjunto de datos (el valor predeterminado) o especificarse mediante el argumento `orden`. Por ejemplo, 
- `orden=[0,1]` primero predeciría la salida 0, luego la primera salida 1
- `orden=[1,0]` primero predeciría la última variable de salida y luego la primera variable de salida. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre [`RegressorChain`](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.RegressorChain.html#sklearn.multioutput.RegressorChain)

</div>

<a id="section51"></a> 
## <font color="#004D7F"> 5.1. Evaluación</font>

El siguiente ejemplo demuestra cómo podemos crear primero un modelo de regresión de salida única y luego usar la clase `RegressorChain` para envolver el modelo de regresión y agregar soporte para la regresión de múltiples salidas.

In [11]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR

# definir modelo base
model = LinearSVR()
# definir el modelo contenedor directo de salida múltiple
wrapper = RegressorChain(model)
# definir el procedimiento de evaluación
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluar el modelo y recoger las puntuaciones
n_scores = cross_val_score(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -0.597 (0.323)


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Tenga en cuenta que es posible que vea una advertencia de convergencia (`ConvergenceWarning`) al ejecutar el ejemplo, que puede ignorarse de forma segura.
</div>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Sus resultados pueden variar dada la naturaleza estocástica del algoritmo o procedimiento de evaluación, o diferencias en la precisión numérica. Considere ejecutar el ejemplo varias veces y comparar el resultado promedio.
</div>

<a id="section52"></a> 
## <font color="#004D7F"> 5.2. Predicción</font>

También podemos utilizar el contenedor de regresión multisalida encadenada como modelo final y hacer predicciones sobre nuevos datos. 
1. Primero, el modelo se ajusta a todos los datos disponibles;
2. Luego se puede llamar a la función `predict()` para hacer predicciones sobre nuevos datos. 

In [12]:
# definir modelo base
model = LinearSVR()
# definir el modelo contenedor directo de salida múltiple
wrapper = RegressorChain(model)
# ajustar el modelo en todo el conjunto de datos
wrapper.fit(X, y)
# hacer una sola predicción
row = [-0.64088429,  0.46455086,  0.55353581, -0.62778801, -0.3466824 ,
        2.97578929,  0.35570561, -0.26769419,  0.58943853, -0.18435229]
yhat = wrapper.predict([row])
print('Predicha: %s' % yhat[0])

Predicha: [-81.70488088  45.70383023]


/Users/javierpiquerasmartinez/miniconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="sectionEj"></a>
<h3><font color="#004D7F" size=6> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicios</font></h3>

Se proponen las siguientes actividades para consolidar el aprendizaje.

# <font color="#004D7F" size=5>Ejercicio 1</font>
__Hiperparámetros__. ¿Qué otros hiperparámetros se pueden utilizar? Indaga en estos para las diferentes clases que hemos visto y evalúe su uso. Recuerda evaluar en todas las clases principales vistas.

### Hiperparámetros de LinearSVR

En `LinearSVR` de scikit-learn, puedes ajustar varios hiperparámetros para optimizar el rendimiento de tu modelo. Aquí tienes una lista de los principales hiperparámetros que puedes utilizar:

1. **`epsilon`**: Parámetro en la función de pérdida epsilon-insensible. Controla la tolerancia a los errores dentro de una banda alrededor de la predicción. Por defecto es `0.0`.
2. **`tol`**: Tolerancia para los criterios de parada. Por defecto es `0.0001`.
3. **`C`**: Parámetro de regularización. La fuerza de la regularización es inversamente proporcional a `C`. Debe ser estrictamente positivo. Por defecto es `1.0`.
4. **`loss`**: Especifica la función de pérdida. Puede ser `'epsilon_insensitive'` (L1) o `'squared_epsilon_insensitive'` (L2). Por defecto es `'epsilon_insensitive'`.
5. **`fit_intercept`**: Indica si se debe ajustar una intersección. Si es `True`, el vector de características se extiende para incluir un término de intersección. Por defecto es `True`.
6. **`intercept_scaling`**: Cuando `fit_intercept` es `True`, este parámetro escala el término de intersección. Por defecto es `1.0`.
7. **`dual`**: Selecciona el algoritmo para resolver el problema de optimización dual o primal. Puede ser `True`, `False` o `'auto'`. Por defecto es `'auto'`.
8. **`verbose`**: Nivel de verbosidad. Si es mayor que `0`, se imprimen mensajes de progreso. Por defecto es `0`.
9. **`random_state`**: Controla la aleatoriedad del estimador. Puede ser un entero o `None`. Por defecto es `None`.
10. **`max_iter`**: Número máximo de iteraciones. Por defecto es `1000`.

# <font color="#004D7F" size=5>Ejercicio 2</font>
__Más conjuntos de datos__. Busque un dataset original y verdadero (que no sea sintético) y evalúe el uso de los conceptos vistos en esta unidad. Los conjuntos de datos en pueden ser obtenidos del [repositorio de aprendizaje automático de UCI](https://archive.ics.uci.edu/).

In [ ]:
from sklearn.metrics import mean_absolute_error
# Recuperar el conjunto de datos linnerud
from sklearn.datasets import load_linnerud
X, y = load_linnerud(return_X_y=True)

# Crear un modelo LinearSVR
model = LinearSVR()
# Crear un modelo contenedor de salida múltiple
wrapper = MultiOutputRegressor(model)

# Evaluar el modelo
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# Almacenar las puntuaciones
scores = []

# Evaluar el modelo
for train_ix, test_ix in cv.split(X):
    X_train, X_test, y_train, y_test = X[train_ix], X[test_ix], y[train_ix], y[test_ix]
    wrapper.fit(X_train, y_train)
    yhat = wrapper.predict(X_test)
    score = mean_absolute_error(y_test, yhat)
    scores.append(score)

# Calcular la media y la desviación estándar de las puntuaciones
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

# Ajustar el modelo en todo el conjunto de datos
wrapper.fit(X, y)

# Hacer una sola predicción
row = [198, 104, 50]
yhat = wrapper.predict([row])
print('Predicha: %s' % yhat[0])




MAE: 48.164 (16.498)
Predicha: [530.66558316  82.99773789 166.90640831]


/Users/javierpiquerasmartinez/miniconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/javierpiquerasmartinez/miniconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/javierpiquerasmartinez/miniconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/javierpiquerasmartinez/miniconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/javierpiquerasmartinez/miniconda3/lib/python3.7/site-packages/sklearn

---

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>